## Supplement 6: Decision Trees and Random Forest

In [43]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from scipy.stats import mode
from sklearn.tree import DecisionTreeRegressor

### 6.3 Programming Task: Song popularity prediction using Random Forest
The goal of this task is to train a random forest model that predicts the song popularity using the datasets already provided in task 4.3
 

In [44]:
# Read data

#TODO
train = pd.read_csv("train-songs.csv").to_numpy()
test = pd.read_csv("test-songs.csv").to_numpy()

train_X = train[:, :-1]
train_y = train[:, -1]
test_X = test[:, :-1]
test_y = test[:, -1]

   i\. Implement a function that draws a bootstrap sample of size N from the train dataset, where N can be specified by the user.




In [45]:
def generate_bootstrap(train_X,train_y,N):
   idx = np.random.randint(len(train_X), size=N)
   return train_X[idx, :], train_y[idx]




   ii\. Complete the implementation of the random forest algorithm. For this task you may use the DecisionTreeClassifier from the scikit-learn library. The other parts of the random forest algorithm must be implemented using only Scipy/Numpy.

In [46]:
class RandomForest:
   def __init__(self,n_trees,max_features,max_samples,min_node_size, max_depth):

      #TODO Initialize list containing weak classifiers. Also initialize any other parameter if required.
      self.trees = []
      self.random_features_of_trees = []
      self.n_trees = n_trees
      self.max_features = max_features
      self.max_samples = max_samples
      self.min_node_size = min_node_size
      self.max_depth = max_depth



   def train(self,train_X,train_y):
      #TODO Training each weak classifier
      original_feature_count = len(train_X[0])

      for i in range(self.n_trees):
         clf = DecisionTreeRegressor(min_samples_leaf=self.min_node_size, max_depth=self.max_depth)
         train_bs, test_bs = generate_bootstrap(train_X, train_y, self.max_samples)
         
         random_features = np.random.choice(original_feature_count, size=self.max_features, replace=False)
         self.random_features_of_trees.append(random_features)

         clf.fit(train_bs[:, random_features], test_bs)
         self.trees.append(clf)


   
   def predict(self,test_X):
      #TODO Final predictions are obtained by taking majority-vote (most frequent class) from each weak classifier prediction
      y_prediction = 0
      
      for i, tree_clf in enumerate(self.trees):
         features = self.random_features_of_trees[i]

         predictions = tree_clf.predict(test_X[:, features])
         y_prediction += predictions

      y_prediction /= self.n_trees
      y_prediction[y_prediction>0.5] = 1
      y_prediction[y_prediction <=0.5] = 0
      return y_prediction




   




iii\. Train the model for the dataset from train-songs.csv using the parameters given below.
| Parameter| Value|
|----------|------|
Number of trees|100|
Maximum features per tree|2|
Bootstrap sample size|20000|
Minimum node size|1|
Maximum tree depth|10|


Note: The bootstrap sample size is the same as train dataset size in this task.


In [47]:
# Note: Run this cell without any changes. The model will train if the implementation of subtask (ii) is correct.

random_forest_model = RandomForest(n_trees=100, max_samples=20000,max_depth=10,min_node_size=1, max_features=2 )

random_forest_model.train(train_X, train_y)




   iv\. Calculate the accuracy of the model using the test dataset and compare your results with the
RandomForestClassifier from the scikit-learn library using the following parameters.

In [48]:
# TODO Run predict for test data and calculate accuracy
predictions = random_forest_model.predict(test_X)

accuracy = 1 - np.mean( np.abs(predictions - test_y) ) 
print("Prediction accuracy of Random Forest: ", accuracy)


Prediction accuracy of Random Forest:  0.8


In [49]:
# TODO: Train and predict using scikit-learn library
clf = RandomForestClassifier(n_estimators=100, max_samples=20000, max_depth=10, min_samples_leaf=1, max_features=2)
clf.fit(train_X, train_y)
clf_pred = clf.predict(test_X)

accuracy_sk = 1 - np.mean( np.abs(clf_pred - test_y) ) 
print("Prediction accuracy of Sklearn Random Forest: ", accuracy)

Prediction accuracy of Sklearn Random Forest:  0.8
